In [1]:
import pandas as pd
import numpy as np
import datetime
import scipy.stats as stats
from scipy.stats import norm, ttest_ind
from tqdm import tqdm

**Задача 4. Численный эксперимент**
  

Мы рассмотрели несколько вариантов добавления эффекта. Есть ли смысл думать о способе добавления эффекта при оценке вероятности ошибки II рода или все способы дают одинаковый результат? Результаты могут быть разными. Чтобы в этом убедиться, проведём численный эксперимент.

Допустим, в наш А/В-тест попадают все пользователи, совершавшие покупки до 28 марта.

 
Целевая метрика — средняя выручка с клиента за время эксперимента. Целевую метрику считаем на неделе с 21 по 28 марта. Уровень значимости — 0.05. Критерий — тест Стьюдента. Размер групп — 1000. Ожидаемый эффект — средняя выручка увеличится на 10%.

Нужно оценить вероятности ошибок II рода для трёх вариантов добавления эффекта:

1. Добавление константы ко всем значениям;

2. Умножение на константу всех значений;

3. Добавление константы к 2.5% значений.

Для решения используйте данные из файла `2022-04-01T12_df_sales.csv`.

In [2]:
df_sales_raw = pd.read_csv('./2022-04-01T12_df_sales.csv')
df_sales_raw.tail(3)

,sale_id,date,count_pizza,count_drink,price,user_id
203844,1203845,2022-04-01 11:59:43,1,0,600,7cdcc7
203845,1203846,2022-04-01 11:59:45,1,1,630,47b825
203846,1203847,2022-04-01 11:59:51,1,0,780,cdaabb


In [3]:
df_sales_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203847 entries, 0 to 203846
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   sale_id      203847 non-null  int64 
 1   date         203847 non-null  object
 2   count_pizza  203847 non-null  int64 
 3   count_drink  203847 non-null  int64 
 4   price        203847 non-null  int64 
 5   user_id      203847 non-null  object
dtypes: int64(4), object(2)
memory usage: 9.3+ MB


In [4]:
df_sales_raw['date'] = pd.to_datetime(df_sales_raw['date'])

df_sales = df_sales_raw[(df_sales_raw['date']>=datetime.datetime(2022, 3, 21))\
                        &(df_sales_raw['date']<datetime.datetime(2022, 3, 28))].copy(deep=True)

df_users = df_sales_raw[(df_sales_raw['date']<datetime.datetime(2022, 3, 28))][['user_id']].drop_duplicates()\
    .reset_index(drop=True).copy(deep=True)

reven_df = df_sales.groupby(['user_id'])['price'].sum().reset_index().rename(columns={'price':'revenue'})
reven_df.shape

(25657, 2)

In [5]:
df = df_users.merge(reven_df, on = ['user_id'], how='left').fillna(0)
df.tail()

,user_id,revenue
98579,418026,600.0
98580,a34b19,780.0
98581,bbd203,660.0
98582,1d973d,600.0
98583,e419fe,780.0


`a, b = np.random.choice(values, (2, sample_size), False)`

из массива values берутся две независимые выборки по sample_size элементов.
Параметр False (эквивалентен replace=False) указывает, что элементы выбираются без повторений.
Результат — два массива (a и b), каждый размером (1000), содержащие случайно выбранные значения из values.


`indexes = np.random.choice(np.arange(sample_size), int(sample_size * 0.025), False)`

Создаётся массив индексов (`np.arange(sample_size)`) - создаёт массив последовательных чисел от 0 до sample_size - 1.
Если sample_size = 1000, то этот массив будет содержать числа [0, 1, 2, ..., 999].
Выбираются случайные индексы (`np.random.choice(np.arange(sample_size), int(sample_size * 0.025), False)`). 
`int(sample_size * 0.025)` вычисляет количество элементов, которые нужно выбрать.
Например, если sample_size = 1000, то 1000 * 0.025 = 25, значит, выберется 25 случайных индексов.
`np.random.choice(..., replace=False)` выбирает случайные уникальные индексы (без повторений).

In [6]:
alpha = 0.05
sample_size = 1000
effect = 0.1

pvalues = {'one':[], 'two':[], 'three':[]}
values = df['revenue'].values
mean = values.mean()

for _ in tqdm(range(5000)):
    #берем 2 независимые выборки
    a, b = np.random.choice(values, (2, sample_size), False)
    #к группе b добавляем эфф-т тремя способами
    #1 Добавление константы ко всем значениям
    b_one = b + mean * effect
    #2. Умножение на константу всех значений;
    b_two = b * (1 + effect)
    #3. Добавление константы к 2.5% значений.
    #отбираем 2.5% значений
    indexes = np.random.choice(np.arange(sample_size), int(sample_size * 0.025), False)
    #вычисляем фиксированное значение
    add_value = effect * mean * sample_size / len(indexes)
    #генерим лист с нулями размера sample_size
    mask = np.zeros(sample_size)
    #добавляем 1 к элементам из маски по рандомным индексам (к которым хотим применить эфф-т)
    mask[indexes] += 1
    #добавляем константу к 2.5% значений
    b_three = b + mask*add_value
    #считаем и сохраняем p-value

    for b_, key in ((b_one, 'one'), (b_two, 'two'), (b_three, 'three', ), ):
        pvalues[key].append(stats.ttest_ind(a, b_).pvalue)
    
#считаем точечные оценки вероятностей ошибки II рода

for key, v in pvalues.items():
    errors = (np.array(v) > alpha).astype(int)
    part_errors = np.mean(errors)
    print(f"{key}: part_errors = {part_errors:0.4f}")


#проверим, что отличия стат значимые
print(stats.ttest_ind(pvalues['one'], pvalues['three']).pvalue)
print(stats.ttest_ind(pvalues['two'], pvalues['three']).pvalue)

100%|██████████| 30000/30000 [05:03<00:00, 98.91it/s] 


one: part_errors = 0.8138
two: part_errors = 0.8280
three: part_errors = 0.8238
0.007377442001213756
0.0159232708502451


### Задача 5. Функция оценки вероятности ошибок I/II рода
   

 реализовать код для оценки вероятностей ошибок I и II рода.

 Напишите функцию `estimate_errors`.

 Обратите внимание, что на вход функции подаются не исходные значения метрики, а генератор, который выдаёт уже сэмплированные данные. Это позволит нам детерминировано протестировать правильноcть решения. Самостоятельно сэмлировать данные внутри функции estimate_errors не нужно.

Пример реализации генератора `group_generator`

In [21]:
def create_group_generator(metrics, sample_size, n_iter):
    """Генератор случайных групп.

    :param metrics (pd.DataFame): таблица с метриками, columns=['user_id', 'metric'].
    :param sample_size (int): размер групп (количество пользователей в группе).
    :param n_iter (int): количество итераций генерирования случайных групп.
    :return (np.array, np.array): два массива со значениями метрик в группах.
    """
    user_ids = metrics['user_id'].unique()
    for _ in range(n_iter):
        a_user_ids, b_user_ids = np.random.choice(user_ids, (2, sample_size), False)
        a_metric_values = metrics.loc[metrics['user_id'].isin(a_user_ids), 'metric'].values
        b_metric_values = metrics.loc[metrics['user_id'].isin(b_user_ids), 'metric'].values
        yield a_metric_values, b_metric_values

In [28]:
import numpy as np
import pandas as pd
from scipy import stats 

def estimate_errors(group_generator, effect_add_type, effect, alpha):
    """Оцениваем вероятности ошибок I и II рода.

    :param group_generator: генератор значений метрик для двух групп.
    :param effect_add_type (str): способ добавления эффекта для группы B.
        - 'all_const' - увеличить всем значениям в группе B на константу (b_metric_values.mean() * effect / 100).
        - 'all_percent' - увеличить всем значениям в группе B в (1 + effect / 100) раз.
    :param effect (float): размер эффекта в процентах.
        Пример, effect=3 означает, что ожидаем увеличение среднего на 3%.
    :param alpha (float): уровень значимости.
    :return pvalues_aa (list[float]), pvalues_ab (list[float]), first_type_error (float), second_type_error (float):
        - pvalues_aa, pvalues_ab - списки со значениями pvalue
        - first_type_error, second_type_error - оценки вероятностей ошибок I и II рода.
    """
    pvalues_aa = []
    pvalues_ab = []
    aa_rejections = 0
    ab_rejections = 0
    total_ab_tests = 0
    total_aa_tests = 0


    for a_metric_values, b_metric_values in group_generator:
        #A/A тест
        t_stat_aa, p_value_aa = stats.ttest_ind(a_metric_values, b_metric_values)
        pvalues_aa.append(p_value_aa)
        if p_value_aa < alpha:
            aa_rejections += 1
        total_aa_tests += 1
        


        #Добавление эффектап в группу B
        if effect_add_type == 'all_const':
            effect_value = b_metric_values.mean() * effect / 100
            b_metric_values = b_metric_values + effect_value
        elif effect_add_type == 'all_percent':
            b_metric_values = b_metric_values * (1 + effect / 100)

        # A/B тест
        t_stat_ab, p_value_ab = stats.ttest_ind(a_metric_values, b_metric_values)
        pvalues_ab.append(p_value_ab)
        if p_value_ab >= alpha:
            ab_rejections += 1
        total_ab_tests += 1

        #Ошибка I рода (ложный прокрас): частота ложных отклонений нулевой гипотезы в A/A тестах
        first_type_error = aa_rejections / total_aa_tests

        #Ошибка II рода (не задетектили эфф-т, хотя должны были): частота ошибочного не отвержения нулевой гипотезы в A/B тестах
        second_type_error = ab_rejections / total_ab_tests

    return pvalues_aa, pvalues_ab, first_type_error, second_type_error

In [29]:
sample_size, n_iter, effect, alpha = 100, 10, 6, 0.05

group_generator = (
    (np.arange(sample_size, dtype=float), np.arange(sample_size, dtype=float) + x,)
    for x in range(n_iter)
)
effect_add_type = 'all_const'
pvalues_aa, pvalues_ab, first_type_error, second_type_error = estimate_errors(
    group_generator, effect_add_type, effect, alpha
)

print(pvalues_aa, pvalues_ab, first_type_error, second_type_error)

[1.0, 0.8076897014876132, 0.626466928391483, 0.46552142375730854, 0.33078290948802747, 0.22442071600176872, 0.14521704796052834, 0.08955122234817349, 0.05260398295958351, 0.02942842378712022] [0.46998888346460643, 0.32717767794727437, 0.21622056020900776, 0.13547667660909338, 0.08040923108323633, 0.0451857170247001, 0.024036471488025152, 0.01210479143057956, 0.005773118838539618, 0.0026089925782345424] 0.1 0.5


In [30]:
group_generator = (
    (np.arange(sample_size, dtype=float), np.arange(sample_size, dtype=float) + x,)
    for x in range(n_iter)
)
effect_add_type = 'all_percent'
pvalues_aa, pvalues_ab, first_type_error, second_type_error = estimate_errors(
    group_generator, effect_add_type, effect, alpha)

print(pvalues_aa, pvalues_ab, first_type_error, second_type_error)

[1.0, 0.8076897014876132, 0.626466928391483, 0.46552142375730854, 0.33078290948802747, 0.22442071600176872, 0.14521704796052834, 0.08955122234817349, 0.05260398295958351, 0.02942842378712022] [0.4831887423209038, 0.3416347700112454, 0.23004311997715893, 0.14734018210283897, 0.08968620318339088, 0.05185562066504247, 0.02847298088463613, 0.014847390182757854, 0.007354593433564279, 0.0034622284148958867] 0.1 0.6
